In [99]:
from dotenv import load_dotenv
import json
from openai import OpenAI
import os
from sklearn.metrics.pairwise import cosine_similarity

load_dotenv()

True

In [43]:
client = OpenAI(
    api_key = os.getenv("RUNPOD_TOKEN"),
    base_url = os.getenv("RUNPOD_BASE_URL")
)
model_name = os.getenv("MODEL_NAME")

In [48]:
messages=[
        {"role": "user", "content": "Why is the capital of Egypt?"}
]


# get llama response

In [49]:
def get_chatbot_response(client, model_name, messages, temperature=0):
    input_messages = []
    for message in messages:
        input_messages.append({"role": message["role"], "content": message["content"]})
    # Create a chat completion
    response = client.chat.completions.create(
        model=model_name,
        messages=input_messages,
        temperature=temperature,
        max_tokens=2000,
    )
    return response.choices[0].message.content

In [ ]:
get_chatbot_response(client, model_name, messages)

# Prompt engineering
## Structured output 

In [51]:
system_prompt = """
You are a helper assistant that answer questions about crispr cas9.
Your output should be in a structured json format exactly like the one the below. You are not allowed to write anything other than json object:
[
    {
        "question": the question that you will answer
        "answer": concise answer for the question
    }
]
"""

In [53]:
messages=[
        {"role": "system", "content": system_prompt}
]

messages.append({"role": "user", "content": "How to design a crispr-cas9 experiment for gene editing?"})

In [ ]:
response = get_chatbot_response(client, model_name, messages)

In [61]:
json_response = json.loads(response)
print(json_response)

[{'question': 'How to design a CRISPR-Cas9 experiment for gene editing?', 'answer': {'steps': ['1. Identify the target gene and its function', '2. Design a guide RNA (gRNA) that specifically binds to the target gene', '3. Choose a suitable Cas9 enzyme and its variant (e.g., SpCas9, AsCas9)', '4. Optimize the gRNA-Cas9 complex design for efficient editing', '5. Select a suitable cell type or organism for editing', '6. Validate the gRNA-Cas9 complex using in vitro assays (e.g., PCR, sequencing)', '7. Perform the CRISPR-Cas9 editing experiment using the optimized gRNA-Cas9 complex', '8. Verify the editing efficiency and specificity using molecular biology techniques (e.g., PCR, sequencing, Western blot)', '9. Analyze the edited cells or organisms for desired phenotypes or functions']}}]


## Input structuring

In [64]:
user_input = """
Get me basic information on different crispr systems:
    ```
        1. cas9
        2. cas12a
        3. prime editor
        4. base editor
    ```
"""
messages=[
        {"role": "system", "content": system_prompt}
]
messages.append({"role": "user", "content": user_input})
response = get_chatbot_response(client, model_name, messages)

In [ ]:
json_response = json.loads(response)
print(json.dumps(json_response, indent=4))

## Give the model time to think (Chain of thought)

In [72]:
user_prompt = """
    calculate the result of this equation: 1 + 3
    Your output should be in a structured json format exactly like below. You are not allowed to write anything else other than json object.

    {
        result: the final number resulted from calculating the equation above.
    
    }
"""

In [73]:
messages=[
        {"role": "user", "content": user_prompt}
]
response = get_chatbot_response(client, model_name, messages)

print(response)

{
    "result": 4
}


In [74]:
user_prompt = """
    calculate the result of this equation: 259 / 2 * 8654 + 91072 * 33 - 12971 
    Your output should be in a structured json format exactly like below. You are not allowed to write anything else other than json object.

    {
        steps: This is where you solve the equation step by step following BEDMAS order of operation. You need to show your work and calculate each step leading to the final result. Feel free to write in free text.
        result: the final number resulted from calculating the equation above.
    
    }
"""

messages=[
        {"role": "user", "content": user_prompt}
]
response = get_chatbot_response(client, model_name, messages)

print(response)

{
    "steps": "First, we calculate the division: 259 / 2 = 129.5. Then, we multiply 129.5 by 8654: 129.5 * 8654 = 112,141.5. Next, we multiply 91072 by 33: 91072 * 33 = 3,003,336. Finally, we add 112,141.5 and 3,003,336, then subtract 12971: 112,141.5 + 3,003,336 = 3,115,477.5, and 3,115,477.5 - 12971 = 3,106,506.5.",
    "result": 3106506.5
}


# RAG - Retrival Augmented Generation

In [ ]:
user_prompt = """
    what is new about primer editors?
"""
messages=[
        {"role": "user", "content": user_prompt}
]
response = get_chatbot_response(client, model_name, messages)
print(response)

In [ ]:
iphone_16 = """
The iPhone 16 introduces several exciting updates, making it one of Apple's most advanced smartphones to date. It features a larger 6.1-inch display for the base model and a 6.7-inch screen for the iPhone 16 Plus, with thinner bezels and a more durable Ceramic Shield. The iPhone 16 Pro and Pro Max boast even larger displays, measuring 6.3 and 6.9 inches respectively, offering the thinnest bezels seen on any Apple product so far.

Powered by the new A18 chip (A18 Pro for the Pro models), these phones deliver significant performance improvements, with enhanced neural engine capabilities, faster GPU for gaming, and machine learning tasks. The camera systems are also upgraded, with the base iPhone 16 sporting a dual-camera setup with a 48MP main sensor. The Pro models offer a 48MP Ultra Wide and 5x telephoto camera, enhanced by Apple’s "Camera Control" button for more flexible photography options.

Apple also introduced advanced audio features like "Audio Mix," which uses machine learning to separate background sounds from speech, allowing for more refined audio capture during video recording. Battery life has been extended, especially in the iPhone 16 Pro Max, which is claimed to have the longest-lasting battery of any iPhone 
9TO5MAC

APPLEMAGAZINE
.

Additionally, Apple has switched to USB-C for faster charging and data transfer, and the Pro models now support up to 2x faster video encoding. The starting prices remain consistent with previous generations, with the iPhone 16 starting at $799, while the Pro models start at $999
"""

messages=[
        {"role": "user", "content": user_prompt}
]
response = get_chatbot_response(client, model_name, messages)
print(response)

### Automatic extraction of context from database

In [80]:
samsung_s23 = """
The Samsung Galaxy S23 brings some incremental but notable upgrades to its predecessor, the Galaxy S22. It features the Snapdragon 8 Gen 2 processor, a powerful chip optimized for the S23 series, delivering enhanced performance, especially for gaming and multitasking. This chip ensures top-tier speed and efficiency across all models, from the base S23 to the larger S23+ and S23 Ultra​
STUFF

TECHRADAR
.

In terms of design, the S23's camera module has been streamlined by removing the raised metal contour around the cameras, creating a cleaner, sleeker look. It also sports the same 6.1-inch 120Hz AMOLED display, protected by tougher Gorilla Glass Victus 2, making it more resistant to scratches and drops​
TECHRADAR
.

The S23 Ultra stands out with its 200MP main camera, offering impressive photo clarity, especially in low-light conditions. The selfie camera across the series has been updated to a 12MP sensor, resulting in sharper selfies. The Ultra model also includes productivity tools such as the S-Pen, which remains an essential feature for note-taking and creative tasks​
STUFF

TECHRADAR
.

Battery life is solid, with the S23 Ultra featuring a 5000mAh battery that lasts comfortably through a day of heavy use. However, charging speeds still lag behind some competitors, with 45W wired charging, which is slower than other brands offering up to 125W charging​
STUFF
.

Overall, the Galaxy S23 series enhances performance, durability, and camera quality, making it a strong contender for users seeking a high-performance flagship.
"""

In [85]:
data = [iphone_16, samsung_s23]
user_prompt = """
    What is new in samsung a10?
"""
embedding_client = OpenAI(
    api_key = os.getenv("RUNPOD_TOKEN"),
    base_url = os.getenv("RUNPOD_EMBEDDING_URL")
)

embedding_model_name = os.getenv("EMBEDDING_MODEL_NAME")

In [87]:
def get_embedding(embedding_client, model_name, text_input):
    output = embedding_client.embeddings.create(
    input=text_input,
    model=model_name
)
    
    embeddings = []
    for embedding in output.data:
        embeddings.append(embedding.embedding)

    return embeddings

In [102]:
output = get_embedding(embedding_client, embedding_model_name, user_prompt)[0]

In [90]:
data_embedding = [get_embedding(embedding_client, embedding_model_name, text)[0] for text in data]

In [103]:
data_similarity = cosine_similarity([output], data_embedding)

In [104]:
data_similarity

array([[0.74482716, 0.52195224]])